# Trace Exporting Example

In [1]:
%load_ext autoreload
%autoreload 2
from pathlib import Path
import sys

# If running from github repo, can use this:
sys.path.append(str(Path().cwd().parent.parent.parent.resolve()))

In [2]:
# Install zipkin python package:
# ! pip install opentelemetry-exporter-zipkin-proto-http

# Start the zipkin docker container:
# ! docker run --rm -d -p 9411:9411 --name zipkin openzipkin/zipkin

# Stop the zipkin docker container:
# ! docker stop $(docker ps -a -q --filter ancestor=openzipkin/zipkin)

In [3]:
from examples.expositional.end2end_apps.custom_app.custom_app import CustomApp
from trulens_eval import Tru
from trulens_eval.tru_custom_app import TruCustomApp

# Need to create dummy endpont for the instrumentation of the DummyAPI to
# happen.
from trulens_eval.feedback.provider.endpoint.dummy import DummyEndpoint
DummyEndpoint()

from opentelemetry.exporter.zipkin.proto.http import ZipkinExporter

tru = Tru(
    otel_exporter=ZipkinExporter(endpoint="http://localhost:9411/api/v2/spans")
)

tru.reset_database()

tru.start_dashboard(
    force=True, _dev=Path().cwd().parent.parent.parent.resolve()
)

✅ OpenTelemetry exporter set: ZipkinExporter
🦑 Tru initialized with db url sqlite:///default.sqlite .
🛑 Secret keys may be written to the database. See the `database_redact_keys` option of `Tru` to prevent this.
Force stopping dashboard ...
killing psutil.Process(pid=40465, name='python3.12', status='running', started='18:46:21')
Starting dashboard ...
Config file already exists. Skipping writing process.
Credentials file already exists. Skipping writing process.


Accordion(children=(VBox(children=(VBox(children=(Label(value='STDOUT'), Output())), VBox(children=(Label(valu…

Dashboard started at http://192.168.1.130:8501 .


<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>

In [4]:
# Create custom app:
ca = CustomApp()

# Create trulens wrapper:
ta = TruCustomApp(
    ca,
    app_id="customapp",
)

In [5]:
# Nested context managers.
with ta as recorder1:
    ca.llm.generate("this is the outer memory remember")
    with ta as recorder2:
        ca.memory.remember("this is the inner memory remember")
        ca.respond_to_query(f"hello")

✅ Exporting 1 spans to ZipkinExporter.
✅ Exporting 15 spans to ZipkinExporter.
✅ Exporting 2 spans to ZipkinExporter.
✅ Exporting 1 spans to ZipkinExporter.
✅ Exporting 15 spans to ZipkinExporter.
